# Note: FTX has been liquidated and no longer exists. The follwing code should be viewed an another reference/showcase for Cryptocurrency API Trading using CCXT. 

# API Trading with FTX

__Remember:__

Install ccxt with:
- pip install ccxt

(first: conda update anaconda)

## Introduction

In [ ]:
import ccxt
import pandas as pd

In [ ]:
print(ccxt.exchanges) # all supported exchanges

- Separate Exchanges for FTX.com & FTX.us
- no separate Exchanges for Spot & Futures Trading (different symbols: BTC/USD vs. BTC-PERP)

__FTX.com__

In [ ]:
exchange = ccxt.ftx()
exchange

In [ ]:
exchange.headers = {"FTX-SUBACCOUNT": "Futures",}

In [ ]:
exchange.fetchTicker(symbol = 'BTC-PERP') # get current market data

__Insert your FTX login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [ ]:
exchange.fetchBalance()["info"]["balances"] # with binance

In [ ]:
exchange.fetchBalance()["info"]["result"] # default: Main Account

In [ ]:
pd.DataFrame(exchange.fetchBalance()["info"]["result"])

__FTX.US__

In [ ]:
ftxus = ccxt.ftxus()
ftxus

In [ ]:
ftxus.fetchTicker(symbol = 'BTC/USDT') # get current market data

## Backtesting with FTX

-> Works in the same way as for any other exchange

-> Most important: Determine the applicable proportional trading costs (ptc) level! (Fees + hidden costs)

-> Approximately, you could even use historical data from other exchanges (as FTX history is limited) 

In [ ]:
import ccxt
import pandas as pd
import time

In [ ]:
exchange = ccxt.ftx()

In [ ]:
def get_history(symbol, interval, start = None, limit = 1000):
    
    if start:
        start = exchange.parse8601(start)
    
    data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
    last_bar_actual = data[-1][0] # timestamp of last loaded bar
    
    # timestamp of current bar 
    last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
    # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
    while last_bar_target != last_bar_actual: 
        
        time.sleep(0.1)
        data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval,
                                      since = last_bar_actual, limit = limit)
        data += data_add[1:]
        last_bar_actual = data[-1][0]
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
    df = pd.DataFrame(data)
    df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
    df.Date = pd.to_datetime(df.Date, unit = "ms")
    df.set_index("Date", inplace = True)

    return df

In [ ]:
start = "2020-08-15 09:00:00"

In [ ]:
df = get_history(symbol = "BTC/USD", interval = "1h", start = start, limit = 1000)
df

In [ ]:
df = get_history(symbol = "BTC/USD", interval = "1d", limit = 1000) # Spot
df

In [ ]:
df = get_history(symbol = "BTC-PERP", interval = "1d", limit = 1000) # Perp Futures
df

## Algorithmic Spot Trading with FTX and CCXT

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with a Spot Testnet and NOT with a Live Trading Account._

In [ ]:
import pandas as pd
import numpy as np
import time
import ccxt
from threading import Thread

In [ ]:
class CCXTSpotTrader(): # based on Long-Short Trader (Contrarian Strategy)
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh,
                 units, position = 0, sandbox = True):
        
        exchange.set_sandbox_mode(sandbox) 
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.get_available_intervals()
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = []
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def get_available_intervals(self):
        
        l = []
        for key, value in exchange.timeframes.items():
            l.append(key)
        self.available_intervals = l
    
    def start_trading(self, start = None, hist_bars = None):
        
        if not hist_bars:
            hist_bars = 1000
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 start = start, limit = hist_bars)
            thread = Thread(target = self.start_kline_stream, args = (self.stream_candles, self.symbol, self.bar_length))
            thread.start()
            
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, start, limit):
        
        if start:
            start = exchange.parse8601(start)
    
        data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
        last_bar_actual = data[-1][0]
    
        # timestamp of current bar
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
        # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
        while last_bar_target != last_bar_actual:
        
            time.sleep(0.1)
            data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = last_bar_actual, limit = limit)
            data += data_add[1:]
            last_bar_actual = data[-1][0]
            last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
        df = pd.DataFrame(data)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        df.Date = pd.to_datetime(df.Date, unit = "ms")
        df.set_index("Date", inplace = True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar = df.index[-1]

        self.data = df
        
    def stream_candles(self, msg):
        # defines how to process the msg
    
        # extract data form msg
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        # if most recent bar is suddenly missing
        if start_time < self.last_bar:
            pass # do nothing and pull the next msg
            
        else:
            if start_time == self.last_bar:
                complete = False
            elif start_time > self.last_bar:
                complete = True
                if len(msg) == 2:
                    self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
                else:
                    self.data.loc[self.last_bar, "Complete"] = complete
                self.last_bar = start_time
            
            # print something
            print(".", end = "", flush = True)
        
            # feed df with latest bar
            self.data.loc[start_time] = [first, high, low, close, volume, False]
        
            # if a bar is complete, define strategy and trade
            if complete == True:
                print("Define Strategy and check Trades!")
                self.define_strategy()
                #self.execute_trades()
    
    def start_kline_stream(self, callback, symbol, interval):
    
        self.running = True
    
        while self.running == True:
            msg = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)
        
            if len(msg) == 0:
                print("No data received")
            else:
                callback(msg)
    
            time.sleep(1)
    
    def stop_stream(self):
        self.running = False     
         
    def define_strategy(self):
        
        df = self.data.loc[self.data.Complete == True].copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns <= self.return_thresh[0]
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        cond3 = df.returns >= self.return_thresh[1]
        
        df["position"] = 0
        df.loc[cond1 & cond2, "position"] = 1
        df.loc[cond3 & cond2, "position"] = -1
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
            
    def report_trade(self, order, going): 
        
        # extract data from order object
        order_id = order["id"]
        time.sleep(0.1)
        
        order = exchange.fetchOrder(id = order_id, symbol = self.symbol)
        
        side = order["side"]
        order_time = pd.to_datetime(order["timestamp"], unit = "ms")
        base_units = float(order["filled"])
        quote_units = float(order["cost"])
        price = float(order["average"])
        
        # calculate trading profits
        self.trades += 1
        if side == "buy":
            self.trade_values.append(-quote_units)
        elif side == "sell":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(order_time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(order_time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(order_time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")
        

In [ ]:
exchange = ccxt.ftx()
exchange

__Copy/Paste your login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [ ]:
symbol = "BTC/USD"
bar_length = "1m"
return_thresh = [-0.0001, 0.0001]
volume_thresh = [-3, 3]
units = 0.0001
position = 0

In [ ]:
trader = CCXTSpotTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = 0, sandbox = False)

In [ ]:
trader.start_trading(start = None, hist_bars = 10)

In [ ]:
trader.stop_stream()

In [ ]:
trader.prepared_data

## How to get access to FTX Subaccounts

In [ ]:
import ccxt
import pandas as pd

In [ ]:
exchange = ccxt.ftx()
exchange

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [ ]:
exchange.fetchBalance()["info"]["result"] # default: Main Account

In [ ]:
exchange.headers = {"FTX-SUBACCOUNT": "your_subaccount_name",} # go to Subaccount

In [ ]:
exchange.headers = {"FTX-SUBACCOUNT": "Futures",} 

In [ ]:
exchange.fetchBalance()["info"]["result"]

In [ ]:
exchange.headers = {} # go back to MainAccount

In [ ]:
exchange.fetchBalance()["info"]["result"]

## Algorithmic Futures Trading with FTX and CCXT

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with a Futures Testnet and NOT with a Live Trading Account._

__Trade Reporting for FTX Futures__<br>
Unlike Binance Futures API, FTX Futures API does __not provide realized PnL for full trades__. <br>
Solution: Use Trade values to calculate realized PnL (see Spot Trading). 

In [ ]:
import pandas as pd
import numpy as np
import time
import ccxt
from threading import Thread

In [ ]:
class CCXTFuturesTrader(): # Based on FuturesTrader (Contrarian)
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh,
                 units, position = 0, leverage = 5, sandbox = True):
        
        exchange.set_sandbox_mode(sandbox)
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.get_available_intervals()
        self.units = units
        self.position = position
        self.leverage = leverage
        self.trades = 0 
        self.trade_values = []
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def get_available_intervals(self):
        
        l = []
        for key, value in exchange.timeframes.items():
            l.append(key)
        self.available_intervals = l
    
    def start_trading(self, start = None, hist_bars = None):
        
        if not hist_bars:
            hist_bars = 1000
        
        exchange.set_leverage(leverage = self.leverage, symbol = self.symbol)
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 start = start, limit = hist_bars)
            thread = Thread(target = self.start_kline_stream, args = (self.stream_candles, self.symbol, self.bar_length))
            thread.start()
            
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, start, limit):
        
        if start:
            start = exchange.parse8601(start)
    
        data = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = start, limit = limit)
        last_bar_actual = data[-1][0]
    
        # timestamp of current bar
        last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]
    
        # as long as we don´t have all bars (most recent): let´s pull the next 1000 bars
        while last_bar_target != last_bar_actual:
        
            time.sleep(0.1)
            data_add = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, since = last_bar_actual, limit = limit)
            data += data_add[1:]
            last_bar_actual = data[-1][0]
            last_bar_target = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)[-1][0]      
    
        df = pd.DataFrame(data)
        df.columns = ["Date", "Open", "High", "Low", "Close", "Volume"]
        df.Date = pd.to_datetime(df.Date, unit = "ms")
        df.set_index("Date", inplace = True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar = df.index[-1]

        self.data = df
        
    def stream_candles(self, msg):
        # defines how to process the msg
    
        # extract data form msg
        start_time = pd.to_datetime(msg[-1][0], unit = "ms")
        first = msg[-1][1]
        high = msg[-1][2]
        low = msg[-1][3]
        close  = msg[-1][4]
        volume = msg[-1][5]
        
        if start_time < self.last_bar:
            pass
            
        else:
            if start_time == self.last_bar:
                complete = False
            elif start_time > self.last_bar:
                complete = True
                if len(msg) == 2:
                    self.data.loc[self.last_bar] = [msg[0][1], msg[0][2], msg[0][3], msg[0][4], msg[0][5], complete]
                else:
                    self.data.loc[self.last_bar, "Complete"] = complete
                self.last_bar = start_time
            
            # print something
            print(".", end = "", flush = True)
        
            # feed df with latest bar
            self.data.loc[start_time] = [first, high, low, close, volume, False]
        
            # if a bar is complete, define strategy and trade
            if complete == True:
                print("Define Strategy and check Trades!")
                self.define_strategy()
                #self.execute_trades()
    
    def start_kline_stream(self, callback, symbol, interval):
    
        self.running = True
    
        while self.running == True:
            msg = exchange.fetchOHLCV(symbol = symbol, timeframe = interval, limit = 2)
        
            if len(msg) == 0:
                print("No data received")
            else:
                callback(msg)
    
            time.sleep(1)
    
    def stop_stream(self):
        self.running = False     
         
    def define_strategy(self):
        
        df = self.data.loc[self.data.Complete == True].copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns <= self.return_thresh[0]
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        cond3 = df.returns >= self.return_thresh[1]
        
        df["position"] = 0
        df.loc[cond1 & cond2, "position"] = 1
        df.loc[cond3 & cond2, "position"] = -1
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING LONG")  
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            elif self.position == -1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "buy", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL") 
            self.position = 0
        if self.prepared_data["position"].iloc[-1] == -1: # if position is short -> go/stay short
            if self.position == 0:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING SHORT") 
            elif self.position == 1:
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING NEUTRAL")
                time.sleep(0.1)
                order = exchange.createMarketOrder(symbol = self.symbol, side = "sell", amount = self.units)
                self.report_trade(order, "GOING SHORT")
            self.position = -1
            
    def report_trade(self, order, going): 
        
        # extract data from order object
        order_id = order["id"]
        time.sleep(0.1)
        
        order = exchange.fetchOrder(id = order_id, symbol = self.symbol)
        
        side = order["side"]
        order_time = pd.to_datetime(order["timestamp"], unit = "ms")
        base_units = float(order["filled"])
        quote_units = float(order["cost"])
        price = float(order["average"])
        
        # calculate trading profits
        self.trades += 1
        if side == "buy":
            self.trade_values.append(-quote_units)
        elif side == "sell":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(order_time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(order_time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(order_time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")
        

In [ ]:
exchange = ccxt.ftx() 
exchange

__Copy/Paste your (Futures subaccount) login credentials here:__

In [ ]:
api_key = "insert here"
secret_key = "insert here"

In [ ]:
exchange.apiKey = api_key
exchange.secret = secret_key

In [ ]:
exchange.headers = {"FTX-SUBACCOUNT": "Futures",} 

In [ ]:
symbol = "BTC-PERP"
bar_length = "1m"
return_thresh = [-0.0001, 0.0001]
volume_thresh = [-3, 3]
units = 0.0001
position = 0
leverage = 2

In [ ]:
trader = CCXTFuturesTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                           volume_thresh = volume_thresh, units = units, position = position,
                           leverage = leverage, sandbox = False)

In [ ]:
trader.start_trading(start = None, hist_bars = 10)

In [ ]:
trader.stop_stream()

In [ ]:
trader.prepared_data